## Pageload & Event Queries
**NOTE**: Data for all pages with external links began 3/29 whereas WPM pages with external links began 3/27 meaning a simple subtraction of WPM from all pages will not produce the same summary total as NOT WPM. Possible solutions: either limit study period to >= 3/29 OR use all page IDs found after we started collecting data and assume the deltas are minor (likely true for PWM, likely relatively true for W but numerically a greater difference).
- v. 1.0.0 2019-04-08
- v. 1.0.1 2019-04-17

In [2]:
# basic setup
# use PySpark YARN kernel
import pyspark
import re
import pyspark.sql
from pyspark.sql import *
import pandas as pd
import matplotlib.pyplot as plt
import hashlib
import os.path
from pyspark.sql.functions import desc
from datetime import timedelta, date

%matplotlib inline
spark_hive = pyspark.sql.HiveContext(sc)


In [3]:
# required to iterate the range of dates
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# set date ranges for all queries
start_date = date(2019, 3, 21)
end_date = date(2019, 4, 22)
date_format = '%Y-%m-%d'
start_date_string = start_date.strftime(date_format)
end_date_string = end_date.strftime(date_format)

## Citation Usage Overview

In [62]:
# show citationusage events by date and type

events_query = """
SELECT to_date(dt) date, event.action, COUNT(*) count
FROM event.citationusage
WHERE wiki = 'enwiki'
AND to_date(dt) >= '{}'
AND to_date(dt) <= '{}'
AND useragent.is_bot = FALSE
GROUP BY to_date(dt), event.action
ORDER BY to_date(dt)
"""

events = spark.sql(events_query.format(start_date_string, end_date_string))
events_rdd = events.rdd
events_df = sqlContext.createDataFrame(events_rdd)
events_pandas = events_df.toPandas()

In [64]:
events_pandas.pivot(index='date', columns='action', values='count')

action,extClick,fnClick,fnHover,upClick
date,,,,
2019-03-21,450241,221977,310889,91402
2019-03-22,1590925,725377,1131859,149622
2019-03-23,1389404,667724,883147,20342
2019-03-24,1531353,750329,1018411,23969
2019-03-25,1774686,782508,1365524,82990
2019-03-26,1729764,737057,1342285,28909
2019-03-27,1693519,728824,1321898,27761
2019-03-28,1639009,705904,1267787,25736
2019-03-29,1522480,666005,1122572,23467


## Pageload Data
#### source: event.citationusagepageload

In [67]:
# basic pageloads overview
pageloads_query = """
SELECT to_date(dt) date, event.action, count(*) count
FROM event.citationusagepageload
WHERE wiki = 'enwiki'
AND to_date(dt) >= '{}'
AND to_date(dt) <= '{}'
AND useragent.is_bot = FALSE
GROUP BY to_date(dt), event.action
ORDER BY to_date(dt)
"""

pageloads = spark.sql(pageloads_query.format(start_date_string, end_date_string))
pageloads_rdd = pageloads.rdd
pageloads_df = sqlContext.createDataFrame(pageloads_rdd)
pageloads_pandas = pageloads_df.toPandas()


In [68]:
pageloads_pandas.pivot(index='date', columns='action', values='count')

action,pageLoad
date,
2019-03-21,14270583
2019-03-22,52417604
2019-03-23,51588547
2019-03-24,56845081
2019-03-25,58315949
2019-03-26,55937993
2019-03-27,55018474
2019-03-28,53933565
2019-03-29,51413672


### data refresh required
The queries below use data from daily SQL queries which are then imported into hive. Tables that require refreshing and date they were last refreshed:
- ryanmax.pages_with_extlinks: 2019-04-15
- ryanmax.projmed_categories: 2019-04-12
- ryanmax.projmed_with_extlinks: 2019-04-15

In [78]:
# daily count of pageloads of WP:M pages with external links
## verified against day-at-a-time version of same query
pgload_wpm_extl_query = """
SELECT to_date(citationusagepageload.dt) date, event.action, count(*) count
FROM event.citationusagepageload, ryanmax.projmed_with_extlinks
WHERE wiki = 'enwiki'
AND event.page_id = projmed_with_extlinks.page_id
AND to_date(citationusagepageload.dt) = to_date(projmed_with_extlinks.dt)
AND to_date(citationusagepageload.dt) >= '{}'
AND to_date(citationusagepageload.dt) <= '{}'
AND useragent.is_bot = FALSE
GROUP BY to_date(citationusagepageload.dt), event.action
ORDER BY to_date(citationusagepageload.dt)
"""

pgload_wpm_extl = spark.sql(pgload_wpm_extl_query.format(start_date_string, end_date_string))
pgload_wpm_extl_rdd = pgload_wpm_extl.rdd
pgload_wpm_extl_df = sqlContext.createDataFrame(pgload_wpm_extl_rdd)
pgload_wpm_extl_pandas = pgload_wpm_extl_df.toPandas()

In [79]:
pgload_wpm_extl_pandas.pivot(index='date', columns='action', values='count')

action,pageLoad
date,
2019-03-27,1625817
2019-03-28,1562904
2019-03-29,1409126
2019-03-30,1204098
2019-03-31,1282420
2019-04-01,1580307
2019-04-02,1614142
2019-04-03,1582776
2019-04-04,1574178


In [84]:
# daily count of pageloads of all pages with external links
## verified against day-at-a-time version of same query
pgload_extl_query = """
SELECT to_date(citationusagepageload.dt) date, event.action, count(*) count
FROM event.citationusagepageload, ryanmax.pages_with_extlinks
WHERE wiki = 'enwiki'
AND event.page_id = pages_with_extlinks.page_id
AND to_date(citationusagepageload.dt) = to_date(pages_with_extlinks.dt)
AND to_date(citationusagepageload.dt) >= '{}'
AND to_date(citationusagepageload.dt) <= '{}'
AND useragent.is_bot = FALSE
GROUP BY to_date(citationusagepageload.dt), event.action
ORDER BY to_date(citationusagepageload.dt)
"""

pgload_extl = spark.sql(pgload_extl_query.format(start_date_string, end_date_string))
pgload_extl_rdd = pgload_extl.rdd
pgload_extl_df = sqlContext.createDataFrame(pgload_extl_rdd)
pgload_extl_pandas = pgload_extl_df.toPandas()

In [82]:
pgload_extl_pandas.pivot(index='date', columns='action', values='count')

action,pageLoad
date,
2019-03-29,50242541
2019-03-30,48728118
2019-03-31,54094263
2019-04-01,56399098
2019-04-02,54304072
2019-04-03,54024178
2019-04-04,53771787
2019-04-05,52672342
2019-04-06,50732159


In [89]:
# top 1000 most visited WP:M pages with external links
top1k_query = """
SELECT event.page_id, page.page_title as title, count(*) count
FROM event.citationusagepageload, enwiki.page
WHERE event.page_id = page.page_id
AND wiki = 'enwiki'
AND event.page_id IN (SELECT page_id FROM ryanmax.projmed_with_extlinks)
AND useragent.is_bot = FALSE
AND to_date(dt) >= '{}'
AND to_date(dt) <= '{}'
GROUP BY event.page_id, page.page_title
ORDER BY count(*) desc
LIMIT 1000
"""

top1k_rdd = sc.emptyRDD()
top1k = spark.sql(top1k_query.format(start_date_string, end_date_string))

# write top1k data to a table for later use
top1k.createOrReplaceTempView("temp_top1k")
sqlContext.sql("DROP TABLE IF EXISTS ryanmax.top1k_med");
sqlContext.sql("CREATE TABLE ryanmax.top1k_med AS SELECT * FROM temp_top1k");

top1k.limit(20).toPandas()

# NOTE: Pages may appear with a title of "None" because the page title data source used here is infrequently updated.
# Use the page_id to look up a title: https://en.wikipedia.org/?curid=56880920


,page_id,title,count
0,43573275,Elizabeth_Holmes,294703
1,41779862,Theranos,118000
2,18079,Leonardo_da_Vinci,82258
3,27546,Sexual_intercourse,75875
4,58911,Measles,73546
5,4488176,Munchausen_syndrome_by_proxy,69389
6,37556,Asperger_syndrome,67278
7,4501,Black_Death,63911
8,791546,Ketogenic_diet,62829
9,52135,Pneumonia,61735


## Event Data

#### source: event.citationusage (limited to sampled pageloads from citationusagepageload)

In [4]:
# daily count of events for WP:M pages with ext links over study period
# limited to events w/ pageload data
sampled_wpm_events_query = """
SELECT to_date(dt) date, event.action, count(*) count
FROM event.citationusage
WHERE wiki = 'enwiki'
AND event.page_id IN (SELECT page_id FROM ryanmax.projmed_with_extlinks WHERE dt LIKE '{}')
AND event.session_token in (
    SELECT event.session_token 
    FROM event.citationusagepageload
    WHERE wiki = 'enwiki'
    AND day = {}
    AND month = {}
    AND year = {}
    AND useragent.is_bot = FALSE
    )
AND day = {}
AND month = {}
AND year = {}
AND useragent.is_bot = FALSE
GROUP BY to_date(dt), event.action
ORDER BY to_date(dt)
"""

sampled_wpm_events_rdd = sc.emptyRDD()
for d in daterange(start_date, end_date):
    dt = str(d.year) + '-' + '{0:02d}'.format(d.month) + '-' + '{0:02d}'.format(d.day) + '%'
    sampled_daily_wpm_events = spark.sql(
        sampled_wpm_events_query.format(dt, d.day, d.month, d.year,d.day, d.month, d.year))
    sampled_wpm_events_rdd = sampled_wpm_events_rdd.union(sampled_daily_wpm_events.rdd)

sampled_wpm_events_merged = sqlContext.createDataFrame(sampled_wpm_events_rdd)
sampled_wpm_events = sampled_wpm_events_merged.toPandas()

In [7]:
# daily count of events for NOT WP:M pages with ext links over study period
# limited to events w/ pageload data
sampled_not_wpm_events_query = """
SELECT to_date(dt) date, event.action, count(*) count
FROM event.citationusage
WHERE wiki = 'enwiki'
AND event.page_id IN (SELECT page_id FROM ryanmax.pages_with_extlinks WHERE dt LIKE '{}')
AND event.page_id NOT IN (SELECT page_id FROM ryanmax.projmed_with_extlinks WHERE dt LIKE '{}')
AND event.session_token in (
    SELECT event.session_token 
    FROM event.citationusagepageload
    WHERE wiki = 'enwiki'
    AND day = {}
    AND month = {}
    AND year = {}
    AND useragent.is_bot = FALSE
    )
AND day = {}
AND month = {}
AND year = {}
AND useragent.is_bot = FALSE
GROUP BY to_date(dt), event.action
ORDER BY to_date(dt)
"""

sampled_not_wpm_events_rdd = sc.emptyRDD()
for d in daterange(start_date, end_date):
    dt = str(d.year) + '-' + '{0:02d}'.format(d.month) + '-' + '{0:02d}'.format(d.day) + '%'
    sampled_daily_not_wpm_events = spark.sql(
        sampled_not_wpm_events_query.format(dt, dt, d.day, d.month, d.year,d.day, d.month, d.year))
    sampled_not_wpm_events_rdd = sampled_not_wpm_events_rdd.union(sampled_daily_not_wpm_events.rdd)

sampled_not_wpm_events_merged = sqlContext.createDataFrame(sampled_not_wpm_events_rdd)
sampled_not_wpm_events = sampled_not_wpm_events_merged.toPandas()

In [8]:
# daily count of events for all pages with ext links over study period
# limited to events w/ pageload data
sampled_events_query = """
SELECT to_date(dt) date, event.action, count(*) count
FROM event.citationusage
WHERE wiki = 'enwiki'
AND event.page_id IN (SELECT page_id FROM ryanmax.pages_with_extlinks WHERE dt LIKE '{}')
AND event.session_token in (
    SELECT event.session_token 
    FROM event.citationusagepageload
    WHERE wiki = 'enwiki'
    AND day = {}
    AND month = {}
    AND year = {}
    AND useragent.is_bot = FALSE
    )
AND day = {}
AND month = {}
AND year = {}
AND useragent.is_bot = FALSE
GROUP BY to_date(dt), event.action
ORDER BY to_date(dt)
"""

sampled_events_rdd = sc.emptyRDD()
for d in daterange(start_date, end_date):
    dt = str(d.year) + '-' + '{0:02d}'.format(d.month) + '-' + '{0:02d}'.format(d.day) + '%'
    sampled_daily_events = spark.sql(
        sampled_events_query.format(dt, d.day, d.month, d.year,d.day, d.month, d.year))
    sampled_events_rdd = sampled_events_rdd.union(sampled_daily_events.rdd)

sampled_events_merged = sqlContext.createDataFrame(sampled_events_rdd)
sampled_events = sampled_events_merged.toPandas()

#### Event Summaries (limited to sampled pageloads)

In [5]:
# WPM:M pages with external links
sampled_wpm_events.groupby(['action']).sum()

,count
action,
extClick,164757
fnClick,171784
fnHover,307664
upClick,5548


In [10]:
# NOT WPM:M pages with external links
sampled_not_wpm_events.groupby(['action']).sum()

,count
action,
extClick,8812440
fnClick,4016948
fnHover,6241335
upClick,145779


In [11]:
# all pages with external links
sampled_events.groupby(['action']).sum()

,count
action,
extClick,8957654
fnClick,4168749
fnHover,6511672
upClick,150693


#### Event Daily Counts (limited to sampled pageloads)

In [6]:
# WPM:M pages with external links
sampled_wpm_events.pivot(index='date', columns='action', values='count')

action,extClick,fnClick,fnHover,upClick
date,,,,
2019-03-27,9734,10240,19020,307
2019-03-28,9809,9743,18307,327
2019-03-29,8429,8511,14972,313
2019-03-30,6266,7357,11734,220
2019-03-31,7322,8365,14018,253
2019-04-01,9749,9649,18655,524
2019-04-02,10176,9622,19335,291
2019-04-03,9625,9796,18947,253
2019-04-04,9665,9188,17144,288


In [15]:
# NOT WPM:M pages with external links
sampled_not_wpm_events.pivot(index='date', columns='action', values='count')

action,extClick,fnClick,fnHover,upClick
date,,,,
2019-03-29,493381,213568,351867,7586
2019-03-30,439327,208318,273163,6158
2019-03-31,481997,236050,314065,7503
2019-04-01,554641,248511,423358,18490
2019-04-02,552139,237538,423598,13549
2019-04-03,548462,234237,417269,8347
2019-04-04,541504,232591,408057,7906
2019-04-05,514797,223549,369539,7685
2019-04-06,458952,219654,277847,6284


In [16]:
# all pages with external links
sampled_events.pivot(index='date', columns='action', values='count')

action,extClick,fnClick,fnHover,upClick
date,,,,
2019-03-29,501810,222079,366839,7899
2019-03-30,445593,215675,284897,6378
2019-03-31,489319,244415,328083,7756
2019-04-01,564390,258160,442013,19014
2019-04-02,562315,247160,442933,13840
2019-04-03,558087,244033,436216,8600
2019-04-04,551169,241779,425201,8194
2019-04-05,523111,231995,384761,7913
2019-04-06,465448,227149,289209,6530


In [85]:
# Total event count for top 1000 viewed WP:M pages with ext links by event type
sum_top1k_events_query = """
SELECT event.action, count(*) count
FROM event.citationusage
WHERE wiki = 'enwiki'
AND event.page_id IN (SELECT page_id FROM ryanmax.top1k_med)
AND to_date(dt) >= '{}'
AND to_date(dt) <= '{}'
AND useragent.is_bot = FALSE
AND event.session_token in (
    SELECT event.session_token 
    FROM event.citationusagepageload
    WHERE wiki = 'enwiki'
    AND to_date(dt) >= '{}'
    AND to_date(dt) <= '{}'
    AND useragent.is_bot = FALSE
    )
GROUP BY event.action
"""

sum_top1k_events = spark.sql(
    sum_top1k_events_query.format(
        start_date_string, end_date_string,
        start_date_string, end_date_string,
    ))
sum_top1k_events_rdd = sum_top1k_events.rdd
sum_top1k_events_df = sqlContext.createDataFrame(sum_top1k_events_rdd)
sum_top1k_events_pandas = sum_top1k_events_df.toPandas()
sum_top1k_events_pandas

,action,count
0,fnHover,204566
1,fnClick,106465
2,upClick,4363
3,extClick,59942


In [120]:
# Total event count for each WP:M class (FA, A, GA, B, C, Start, Stub) by event type
## query still needs verification
pm_category_events_query = """
SELECT projmed_categories.category, event.action, count(*) count
FROM 
    event.citationusage, 
    (SELECT DISTINCT page_id, category 
    FROM ryanmax.projmed_categories 
    WHERE projmed_categories.category LIKE '%Class_medicine_articles%') 
    AS projmed_categories
WHERE event.page_id = projmed_categories.page_id
    AND wiki = 'enwiki'
    AND event.page_id IN (SELECT page_id FROM ryanmax.projmed_with_extlinks)
    AND to_date(dt) >= '{}'
    AND to_date(dt) <= '{}'
    AND useragent.is_bot = FALSE
    AND event.session_token in (
        SELECT event.session_token 
        FROM event.citationusagepageload
        WHERE wiki = 'enwiki'
        AND to_date(dt) >= '{}'
        AND to_date(dt) <= '{}'
        AND useragent.is_bot = FALSE
        )
GROUP BY projmed_categories.category, event.action
ORDER BY projmed_categories.category, event.action
"""

pm_category_events = spark.sql(
    pm_category_events_query.format(
        start_date_string, end_date_string,
        start_date_string, end_date_string,
    ))
pm_category_events_rdd = pm_category_events.rdd
pm_category_events_df = sqlContext.createDataFrame(pm_category_events_rdd)
pm_category_events_pandas = pm_category_events_df.toPandas()
# set precision of count values so they don't appear with a decimal place ... likely an easier way to do this
pm_category_events_pandas['count'] = pm_category_events_pandas['count'].map(lambda x: '{0:.0f}'.format(x))
pm_category_events_pandas.pivot(index='category', columns='action', values='count')

action,extClick,fnClick,fnHover,upClick
category,,,,
B-Class_medicine_articles,54169,88301,167946,3943
C-Class_medicine_articles,58593,64867,115333,3555
Category-Class_medicine_articles,5,3,10,None
Disambig-Class_medicine_articles,48,6,9,1
FA-Class_medicine_articles,4056,7837,19328,208
FL-Class_medicine_articles,118,262,331,13
GA-Class_medicine_articles,6901,16979,38430,620
List-Class_medicine_articles,3504,2784,4732,40
Redirect-Class_medicine_articles,39,28,40,1


In [7]:
# count of pages with external links for each WP:M class (FA, A, GA, B, C, Start, Stub)
# numbers will not match [1] because we're limiting to namespace 0 pages with external links
# [1] https://en.wikipedia.org/wiki/Wikipedia:WikiProject_Medicine/Assessment#Statistics
pm_category_pages = """
SELECT category, COUNT(DISTINCT page_id) AS pages_w_links 
FROM ryanmax.projmed_categories 
WHERE category LIKE '%Class_medicine_articles%' 
AND page_id IN (SELECT page_id FROM ryanmax.projmed_with_extlinks)
GROUP BY category
ORDER BY COUNT(*) DESC
"""
pm_cat_counts = spark.sql(pm_category_pages)
cats = sqlContext.createDataFrame(pm_cat_counts.rdd)
cats.toPandas()

,category,pages_w_links
0,Start-Class_medicine_articles,14493
1,Stub-Class_medicine_articles,9824
2,C-Class_medicine_articles,5308
3,B-Class_medicine_articles,2166
4,List-Class_medicine_articles,456
5,GA-Class_medicine_articles,240
6,FA-Class_medicine_articles,62
7,Disambig-Class_medicine_articles,17
8,Redirect-Class_medicine_articles,14
9,FL-Class_medicine_articles,12


In [203]:
# Total event count for top 1K hostnames (e.g., DOI.org / ncbi…) by event type
# limited to W pages with external links
# NOTE: limited to 100 by event count ... do we really need ~4k rows here?
top_hosts_query = """
SELECT parse_url(event.link_url,'HOST') AS host, event.action, COUNT(*) AS count 
FROM event.citationusage 
WHERE wiki = 'enwiki'
AND event.page_id IN (SELECT page_id FROM ryanmax.pages_with_extlinks)
AND to_date(dt) >= '{}'
AND to_date(dt) <= '{}'
AND useragent.is_bot = FALSE
AND event.session_token in (
    SELECT event.session_token 
    FROM event.citationusagepageload
    WHERE wiki = 'enwiki'
    AND to_date(dt) >= '{}'
    AND to_date(dt) <= '{}'
    AND useragent.is_bot = FALSE
    )
GROUP BY host, event.action
ORDER BY COUNT(*) DESC
LIMIT 100
"""
top_hosts_events = spark.sql(
    top_hosts_query.format(
        start_date_string, end_date_string,
        start_date_string, end_date_string,
    ))
top_hosts_events_rdd = top_hosts_events.rdd
top_hosts_events_df = sqlContext.createDataFrame(top_hosts_events_rdd)
top_hosts_events_pandas = top_hosts_events_df.toPandas()
top_hosts_events_pandas

,host,action,count
0,en.wikipedia.org,fnHover,9706012
1,en.m.wikipedia.org,fnClick,4239564
2,en.wikipedia.org,fnClick,2142396
3,www.imdb.com,extClick,823320
4,en.m.wikipedia.org,fnHover,563698
5,web.archive.org,extClick,557417
6,tools.wmflabs.org,extClick,286136
7,en.wikipedia.org,upClick,269190
8,www.youtube.com,extClick,163402
9,books.google.com,extClick,161377
